<a href="https://colab.research.google.com/github/anokhina-rgb/Multilingual-Corpus-for-EU-Studies/blob/main/mwus_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install stanza lxml

import stanza
import xml.etree.ElementTree as ET
import pandas as pd
import re
from IPython.display import display
import os
from google.colab import files

# Download English and Ukrainian models
stanza.download('en')
stanza.download('uk')

# Initialize NLP pipelines
nlp_en = stanza.Pipeline('en', processors='tokenize,mwt,pos,lemma')
nlp_uk = stanza.Pipeline('uk', processors='tokenize,mwt,pos,lemma')

# Upload two .txt files (with en/uk in names)
print("📤 Upload your English and Ukrainian .txt files")
uploaded = files.upload()
filepaths = list(uploaded.keys())

# Identify files
en_file = [f for f in filepaths if 'en' in f.lower()][0]
uk_file = [f for f in filepaths if 'uk' in f.lower()][0]

with open(en_file, encoding='utf-8') as f:
    en_lines = [line.strip() for line in f if line.strip()]

with open(uk_file, encoding='utf-8') as f:
    uk_lines = [line.strip() for line in f if line.strip()]

# Align
min_len = min(len(en_lines), len(uk_lines))
en_lines = en_lines[:min_len]
uk_lines = uk_lines[:min_len]

print(f"✅ Aligned {min_len} sentence pairs")

# MWU extractor: noun/adjective phrases
def extract_mwus(doc):
    mwus = []
    for sent in doc.sentences:
        tokens = sent.words
        for i in range(len(tokens) - 1):
            t1, t2 = tokens[i], tokens[i+1]
            if (t1.upos in ['ADJ', 'NOUN'] and t2.upos == 'NOUN'):
                phrase = f"{t1.text} {t2.text}"
                mwus.append(phrase.lower())
    return mwus

# Process all lines
records = []
for i, (en, uk) in enumerate(zip(en_lines, uk_lines)):
    doc_en = nlp_en(en)
    doc_uk = nlp_uk(uk)
    mwus_en = extract_mwus(doc_en)
    mwus_uk = extract_mwus(doc_uk)
    records.append({
        "EN": en,
        "UK": uk,
        "EN_MWUs": '; '.join(mwus_en),
        "UK_MWUs": '; '.join(mwus_uk),
    })

df = pd.DataFrame(records)
display(df.head())

# Save to TSV (for debugging)
df.to_csv("aligned_mwus.tsv", sep="\t", index=False)

# Build Lexonomy XML
root = ET.Element("dictionary")
for i, row in df.iterrows():
    entry = ET.SubElement(root, "entry")

    head = ET.SubElement(entry, "head")
    head.text = f"{row['EN']}"

    body = ET.SubElement(entry, "body")

    trans = ET.SubElement(body, "translation")
    trans.text = row["UK"]

    mwus = ET.SubElement(body, "mwus")
    mwus_en = ET.SubElement(mwus, "en")
    mwus_en.text = row["EN_MWUs"]

    mwus_uk = ET.SubElement(mwus, "uk")
    mwus_uk.text = row["UK_MWUs"]

# Write XML to file
tree = ET.ElementTree(root)
tree.write("lexonomy_mwus.xml", encoding="utf-8", xml_declaration=True)

# Offer download
files.download("lexonomy_mwus.xml")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 36.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: uk (Ukrainian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/uk/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


...

📤 Upload your English and Ukrainian .txt files


Saving en_1.txt to en_1.txt
Saving ukr_1.txt to ukr_1.txt
✅ Aligned 10 sentence pairs


,EN,UK,EN_MWUs,UK_MWUs
0,The United States Supreme Courtis holding a he...,"У четвер Верховний суд США розглядає справу, щ...",biggest controversy; executive orders; birthri...,верховний суд; скасування права
1,"On his first day as president, the Republican ...",У свій перший день на посаді президента Трамп ...,first day; undocumented immigrant; immigrant p...,перший день; посаді президента; громадянство д...
2,The Supreme Court does not have to rule direct...,Хоча Верховний суд не розглядатиме безпосередн...,court rulings; executive order; entire country,верховний суд; конституційність указу; судові ...
3,The Trump administration asked the Supreme Cou...,Адміністрація Трампа звернулася до Верховного ...,federal judges; executive order,верховного суду; федеральні судді; виконавчого...
4,The administration is asking judges to limit t...,"Адміністрація наполягає на тому, щоб судді обм...",court rulings; lower courts; presidential order,судових рішень; нижчі суди; президентський указ


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>